## Import Requirements
First things first :)

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

## Get a URL

Lets try on my own

In [ ]:
url = "https://www.linkedin.com/in/kaushikdey647/" # url to scrape

head = {
    "User-agent": "KayDee647Scraper",
    "Connection": "keep-alive"
}

response = requests.get(url, headers=head) # get the response from the url
soup = bs(response.text,"html") # parse the response
soup

# Oh no!
Seems Like I can't do it that way, have to find another way around


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

## Initiate WebDrivers
Les go for Chrome 102


In [16]:
driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.linkedin.com/login')

username = "dipadey213@gmail.com"
password = "dipa9864d$#"

/tmp/ipykernel_10104/3447612866.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


In [19]:
from bs4 import BeautifulSoup as bs

title = driver.title
if 'Login' in title:
    print('Login page found')
if 'Feed' in title:
    print('Feed page found')
if 'Verification' in title:
    print('Verification page found')
driver.current_url

Feed page found


'https://www.linkedin.com/feed/'

In [17]:

driver.find_element_by_id("username").send_keys(username)
# find password input field and insert password as well
driver.find_element_by_id("password").send_keys(password)
# Log in dude
driver.find_element_by_xpath("//button[@type='submit']").click()

/tmp/ipykernel_10104/4151366165.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("username").send_keys(username)
/tmp/ipykernel_10104/4151366165.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("password").send_keys(password)
/tmp/ipykernel_10104/4151366165.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//button[@type='submit']").click()


## Lets crack a profile open
Starting with mine ofcourse

In [125]:
driver.get('https://www.linkedin.com/in/shivam-soni-b46992148/')


In [126]:
# wait for the page to load

import time
start = time.time()

#Scroll through the page to load all data
scrollVal = 0

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scrollVal += 1000
    if scrollVal > 10000:
        break

In [18]:
from bs4 import BeautifulSoup as bs

src = driver.page_source
soup = bs(src, 'html.parser')
profile = soup.find(id='profile-content').find(id='main')

In [19]:
left_panel = profile.find('div',{'class': 'pv-text-details__left-panel'})
right_panel = profile.find('ul',{'class': 'pv-text-details__right-panel'})
loc_panel = profile.find('div',{'class': 'pb2 pv-text-details__left-panel'})


name = left_panel.find('h1').get_text().strip() # get the name
try:
    description = left_panel.find('div',{'class': 'text-body-medium break-words'}).get_text().strip() # get the description
except:
    description = "No description"
print(name)
tags = description.split('|')
print(tags)
try:
    workplace = right_panel.find('div',{'aria-label': 'Current company'}).get_text().strip() # get the workplace
except:
    workplace = 'Not Available'
print(workplace)
try:
    education = right_panel.find('div',{'aria-label': 'Education'}).get_text().strip() # get the education
except:
    education = 'Not Available'
print(education)
try:
    location = loc_panel.find('span',{'class': 'text-body-small inline t-black--light break-words'}).get_text().strip() # get the location
except:
    location = 'Not Available'
print(location)

SHIVAM SONI
['Audio AI Engineer @HCL ', ' M.Tech @IIT BBSR ', ' R&D Specialist ', ' Speech & Non-Speech Recognition ', ' Audio Processing ', ' Speech Assistant ', ' Audio Event Detection ', ' Machine & Deep Learning ', ' NLP ', ' Data Science ', ' Kaldi I']
HCL Technologies
Indian Institute of Technology Bhubaneswar (IIT Bhubaneswar)
Noida, Uttar Pradesh, India


## We started gettin the data now
Lets go get more, the about section

In [77]:
about = profile.find('div',{'id':'about'}).parent
about_text = about.find('div',{'class': 'inline-show-more-text inline-show-more-text--is-collapsed'}).get_text().strip()
print(about_text)


-------------------------------------------------- Mentorship ---------------------------------- -------------------------------------------------------- I love to guide students and graduates regarding their careers, research, and college selection. Drop me a message for any kind of assistance. ---------------------------------------------------Difference Maker----------------------------------------------------------------------------------A passionate "Audio AI Research Engineer" with an M.Tech degree in "Signal Processing" from "IIT Bhubaneswar".  Currently, I am working as an Audio AI  Engineer at the HCL Engineering R&D Services.# Potential Areas/ Area of Interest:----------   ---------------  ----------     ---------------- Hands-on Experience ------------------     ------------------   -------------------------1. Digital Signal Processing2. Speech Processing 3. Audio Processing4. Speech Analytics5. Automatic Speech Recognition (ASR) (Kaldi: Aspire, CSJ, Librispeech, Minilibrisp

Moving on to experience

In [78]:

experience = profile.find('div',{'id':'experience'}).parent

exp_list = []
for li in experience.find('ul').find_all('li',recursive=False):
    exp = {}
    try:
        exp['position'] = {}
        exp['position']['title'] = li.find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        subtitle = li.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'}).get_text().strip().split(' · ')

        exp['company/institute'] = subtitle[0]
        try:
            exp['position']['position_type'] = subtitle[1]
        except:
            exp['position']['position_type'] = 'Not Available'
        time_n_place = li.find_all('span',{'class':'t-14 t-normal t-black--light'})
        exp['total_duration'] = time_n_place[0].find('span',{'aria-hidden':'true'}).get_text().strip().split(' · ')[0]
        try:
            exp['location'] = time_n_place[1].find('span',{'aria-hidden':'true'}).get_text().strip()
        except:
            exp['location'] = 'Not Available'
        try:
            exp['position']['description'] = li.find('div',{'class':'pvs-list__outer-container'}).get_text().strip()
        except:
            pass
    except:
        exp['company/institute'] = li.find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        exp['total_duration'] = li.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        try:
            exp['location'] = li.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        except:
            pass
        exp['position'] = []
        for li2 in li.find('ul',{'class':'pvs-list'}).find_all('li',recursive=False):
            position = {}
            position['title'] = li2.find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
            try:
                position['position_type'] = li2.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'}).get_text().strip()
            except:
                position['position_type'] = 'Not Available'
            time_n_place = li.find_all('span',{'class':'t-14 t-normal t-black--light'})
            try:
                position['duration'] = time_n_place[0].find('span',{'aria-hidden':'true'}).get_text().strip().split(' · ')[0]
            except:
                position['duration'] = 'Not Available'
            try:
                exp['description'] = li2.find('div',{'class':'pvs-list__outer-container'}).get_text().strip()
            except:
                pass
            exp['position'].append(position)
    exp_list.append(exp)
exp_list

[{'position': {'title': 'Senior Software Engineer',
   'position_type': 'Full-time',
   'description': 'I am currently working in Speech and Audio AI domain in the HCL Technologies R&D lab. My key responsibilities are audio/speech engine architecture designing, innovating novel audio algorithms, proposing innovative machine learning solutions, identify key research questions, defining data requirements, data collection, data preparation for AI models, project documentation, ASR training on Kaldi speech engine,   performance tuning, acceptance testing, analyzing results, deploying solutions, reading current research papers, publish the result (White paper and research paper writing), and mentoring junior members.My most recent work includes Audio event classification, Educational Videos Summarization, Sentiment Analysis, Cough Classification, Japanese Slurred Voice Assistant for Elderly people, Limited Vocab Indian English Voice Assistant for Touchfree Elevator,  Speech-based Emotion Re

Lets see recommendations now

In [20]:
reccomendationList = profile.find('div',{'id':'recommendations'}).parent
recc_list = []

for li in reccomendationList.find('ul').find_all('li',recursive=False):
    recc = {}
    recc['giver'] = li.find('div',{'class':'display-flex flex-column full-width align-self-center'}).find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    recc['desc'] = li.find('div',{'class':'pvs-list__outer-container'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    recc_list.append(recc)

recc_list

[{'giver': 'Simhadri Vadrevu',
  'desc': 'He was working in the Automatic Audio Event Recognition project and developed various Schemes for Context-Aware Audio Computing Devices. He helped me and other lab members in the department whenever we had problems in the model design. I recall that he had very few related audio recordings and improper model specifications for audio event recognition but he put tremendous effort to develop the Schemes for Context-Aware Audio Computing Devices. He is skillful in his thinking and finding a solution to any problem. He has worked in many tools and is proficient in Matlab, python, c. He is also interested in doing research in Matlab. His involvement with other research scholars investigating different problems related to signal processing is phenomenal. He is a very talented and dedicated person. I strongly recommend him for a Research engineer position. His ethics and commitment towards work will be an asset to any company.'}]

### Lets get them awards finally

In [128]:
awards = profile.find('div',{'id':'honors_and_awards'}).parent
awards_list = []
for li in awards.find('ul').find_all('li',recursive=False):
    award = {}
    award['title'] = li.find('div',{'class':'display-flex align-items-center'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    issuer_n_date = li.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'}).get_text().strip().replace('Issued by ','').split(' · ')
    award['issuer'] = issuer_n_date[0]
    award['date'] = issuer_n_date[1]
    awards_list.append(award)
awards_list

[{'title': 'AICTE PG Scholarship',
  'issuer': 'All India Council for Technical Education (AICTE)',
  'date': 'Sep 2017'},
 {'title': 'Certificate of Merit',
  'issuer': 'Sri Ramswaroop Memorial College of Engineering and Management Lucknow',
  'date': 'Jul 2016'},
 {'title': 'Certificate of Appreciation',
  'issuer': 'Sri Ramswaroop Memorial College of Engineering and Management Lucknow',
  'date': 'Sep 2014'}]

In [42]:
skills = profile.find('div',{'id':'skills'}).parent
skills_list = []
for li in skills.find('ul').find_all('li',recursive=False):
    skill = {}
    skill['name'] = li.find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    skill['endorsers'] = []
    
    skills_list.append(skill)
skills_list
    

['Data Analytics', 'Python (Programming Language)', 'C++']

And the education section

In [43]:
education = profile.find('div',{'id':'education'}).parent

edu_list = []
for li in education.find('ul').find_all('li',recursive=False):
    edu = {}
    edu['institute'] = li.find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    edu['degree'] = li.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    edu['time_period'] = li.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'}).get_text().strip()
    edu_list.append(edu)
edu_list

[{'institute': 'Indian Institute of Technology Bhubaneswar (IIT Bhubaneswar)',
  'degree': 'Bachelor of Technology - B.Tech',
  'time_period': '2018 - 2022'}]

In [129]:
certifications = profile.find('div',{'id':'licenses_and_certifications'}).parent
cert_list = []

for li in certifications.find('ul').find_all('li',recursive=False):
    cert = {}
    try:
        try:
            cert['name'] = li.find('span',{'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        except:
            cert['name'] = li.find('span',{'class':'mr1 t-bold'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        cert['issuer'] = li.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'}).get_text().strip()
        cert_list.append(cert)
    except:
        pass
cert_list

[{'name': 'LISPRUL-2022 (Linguistic Issues in Speech Processing Research of Under-Resourced Languages)',
  'issuer': 'Student Mentorship Program, IIT Dharwad'},
 {'name': 'RECENT TRENDS IN SPEECH ASSISTIVE TECHNOLOGIES',
  'issuer': 'Sri Sivasubramaniya Nadar College Of Engineering'},
 {'name': 'RESEARCH ACCELERATOR',
  'issuer': 'Admissions - Shiv Nadar University, Delhi - NCR'}]

## NOW Lets see how to use GUI to import files and stuff

In [ ]:
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()
file_path = None
while not file_path:
    try:
        file_path = filedialog.askopenfilename()
    except:
        pass
print(file_path)

In [ ]:
from tqdm import tqdm
for line in tqdm(range(100000000),desc="Scraping data"):
    pass

In [ ]:
import scraper

prof = scraper.scrape_profile('https://www.linkedin.com/in/sai-krishna-jupally-b7050177/',driver)
prof

## Lets try multiprocessing now
Lets run two operations in parallel

In [ ]:
import multiprocessing

def init_driver():
    driver = webdriver.Chrome('./chromedriver')
    driver.get('https://www.linkedin.com/login')
    driver.find_element_by_id("username").send_keys(username)
    driver.find_element_by_id("password").send_keys(password)
    driver.find_element_by_xpath("//button[@type='submit']").click()

P1 = multiprocessing.Process(target=init_driver,args=())
P2 = multiprocessing.Process(target=init_driver,args=())
P3 = multiprocessing.Process(target=init_driver,args=())
P4 = multiprocessing.Process(target=init_driver,args=())
P1.start()
P2.start()
P3.start()
P4.start()
P1.join()
P2.join()
P3.join()
P4.join()

In [ ]:
import multiprocessing
def split(lists, process_count):
    k, m = divmod(len(lists), process_count)
    return (lists[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(process_count))

threads = multiprocessing.cpu_count()
urls = [12,124,3256,864,12,643,3856,354547,57,54,547,747,2,57,547,547,54,54,54754,5475,56,8,5427,547,472,547,547,54,547,4757,54,54,547,547]
print(len(urls))
chunks = split(urls,threads//2)

chunks = list(chunks)
chunks


In [ ]:
str = 'https://www.linkedin.com/in/omkar-sawant-836638187/'
parts = str.split('/')
while "" in parts:
    parts.remove("")
print(parts)

In [ ]:
multiprocessing.cpu_count()

In [23]:
import pandas as pd

df = pd.read_csv('./sample_csv/US_Architect_270_QL.csv')

df['Subject Line 1'].iloc[0]

'Best 2019/2020 AIAA SATC Professional Paper Award. (Space Habitat Reconfigurability: TESSERAE platform for self-aware assembly)'

In [34]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import pickle

    

driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.linkedin.com/login')

username = "kaushikdey647@gmail.com"
password = "maihulimkedinka14"


driver.find_element_by_id("username").send_keys(username)
# find password input field and insert password as well
driver.find_element_by_id("password").send_keys(password)
# Log in dude
driver.find_element_by_xpath("//button[@type='submit']").click()

# pickle.dump( driver.get_cookies() , open("cookies.pkl","wb"))

/tmp/ipykernel_10486/3434684360.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')
/tmp/ipykernel_10486/3434684360.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("username").send_keys(username)
/tmp/ipykernel_10486/3434684360.py:20: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("password").send_keys(password)
/tmp/ipykernel_10486/3434684360.py:22: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//button[@type='submit']").click()


In [12]:
from bs4 import BeautifulSoup as bs

def is_verification(soup):
    if soup.find('title').get_text() == 'Security Verification | LinkedIn':
        return True
    else:
        return False


429


In [13]:
src = driver.page_source
soup = bs(driver.page_source, 'html.parser')
if '429' in soup.find('div',{'class':'error-code'}).get_text():
    print('429')

429


In [14]:

is_verification(soup)

True

In [33]:
import pickle

src = driver.page_source
soup = bs(src, 'html.parser')
print(is_verification(soup))
if  is_verification(soup):
    #Verify Manually First and wait for prompt
    pass
else:
    pickle.dump(driver.get_cookies() , open("cookies.pkl","wb"))

False


In [35]:

src = driver.page_source
soup = bs(src, 'html.parser')
if is_verification(soup):
    #Verify Manually First and wait for prompt
    cookies = pickle.load(open("cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    driver.get('https://www.linkedin.com/')

In [45]:
__name__

'__main__'

## PHASE II
Now we use the data collected so far to develop meaningful sentences

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import pickle

    

driver = webdriver.Chrome('./chromedriver')
driver.get('https://www.linkedin.com/login')

username = "kaushikdey647@gmail.com"
password = "maihulimkedinka14"


# driver.find_element_by_id("username").send_keys(username)
# # find password input field and insert password as well
# driver.find_element_by_id("password").send_keys(password)
# # Log in dude
# driver.find_element_by_xpath("//button[@type='submit']").click()


/tmp/ipykernel_10036/2724054809.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [6]:
import scraper
url = 'https://www.linkedin.com/in/shivam-soni-b46992148/'
shivam = scraper.scrape_profile(url,driver)

In [8]:
shivam

{'name': 'SHIVAM SONI',
 'intro': 'Audio AI Engineer @HCL | M.Tech @IIT BBSR | R&D Specialist | Speech & Non-Speech Recognition | Audio Processing | Speech Assistant | Audio Event Detection | Machine & Deep Learning | NLP | Data Science | Kaldi I',
 'intro-education': 'Indian Institute of Technology Bhubaneswar (IIT Bhubaneswar)',
 'intro-work': 'HCL Technologies',
 'intro-location': 'Noida, Uttar Pradesh, India',
 'about': '-------------------------------------------------- Mentorship ---------------------------------- -------------------------------------------------------- I love to guide students and graduates regarding their careers, research, and college selection. Drop me a message for any kind of assistance. ---------------------------------------------------Difference Maker----------------------------------------------------------------------------------A passionate "Audio AI Research Engineer" with an M.Tech degree in "Signal Processing" from "IIT Bhubaneswar".  Currently, I 

In [4]:
import random

strings = {}



#Only use if location is provided
strings['location'] = [
    'I heard you\'re from {location}, how\'s the weather there? ',
    'I heard you\'re from {location}! Been there myself, its a lovely place. ',
    'I can see you\'re from {location}. It\'s a beautiful place. ',
]

#Only use if skill is provided
strings['skill'] = [
    'I have been looking at your profile. I believe you are skilled enough at {skill}. That must have taken a lot of hours ',
    'Looking at your profile, I could see you have a lot of experience with {skill}. How did you get so good at it?',
    'I can see that you are very skilled in {skill}. When did you start working on it?', # Multiple skills
]

strings['awards'] = [
    'Congratulations for your {award}, it is a feat not everyone can achieve... More power to you!',
    'I can also see that you have been awarded {award}! Congratulations to you! How would you describe the joy of the moment you recieved it?',
]

#Only use if certification is provided
strings['certifications'] = [
    'I can also see that you are certified in {certificate} by {issuer}. That is quite a feat! Kudos to you...',
    'I can see that you have a highly valuable {issuer} certification in {certificate}. Congratulations to you!',
]

strings['recommendations'] = [
    'I can see that you have been recommended by {giver} for your skills. That is quite a feat! Kudos to you...',
    'I can see in your profile that {giver} had quite a time working with you. I\'m sure ou must be a great person to work with',
    'I can see that you have been recommended by {giver} on your profile. You have really great connections!',
]

#Only use if experience is provided
strings['experience'] = [
    'I can see that you have experience as a {position} at {company}. How was the experience?',
    'Being a {position} at {company} must have been quite an experience for you.',
    'I can see that you have worked at {company} for the period of {total_time}. How was the experience?',
]

strings['college'] = [
    'I can see that you are an alumni of {college}. That is something to be proud of... More power to you!',
    'Being an alumni of the presitgious {college} is something that sets you apart from the crowd. Keep crushing it!',
    'It is a delight to know that you were a student of {college}. ',
    'It is a great honor to know that you did your {degree} graduated from {college}.',
]

def createTemplate(data,strings = strings):
    '''
    Creates a template from the strings dictionary
    '''
    random.seed()
    template = ''
    #randomly select from list
    template += random.choice(['Hi', 'Greetings', 'Hello', 'Hey']) + ' '
    template += data['name'].split(' ')[0] + ', '
    key = random.choice( list(strings.keys()) )
    print('Key: ', key, '\n')
    str = random.choice(strings[key])
    try:
        if key == 'location':
            str = str.replace('{location}',data['intro-location'].split(',')[0])
        elif key == 'skill':
            str = str.replace('{skill}',random.choice(data['skills']))
        elif key == 'awards':
            award = random.choice(data['awards'])
            str = str.replace('{award}', award['title']).replace('{issuer}', award['issuer'])
        elif key == 'recommendations':
            recc = random.choice(data['recommendations'])
            str = str.replace('{giver}',recc['giver'])
        elif key == 'certifications':
            cert = random.choice(data['licenses_and_certifications'])
            str = str.replace('{certificate}',cert['name']).replace('{issuer}',cert['issuer'])
        elif key == 'experience':
            exp = random.choice(data['experience'])
            try:
                str = str.replace('{position}',exp['position']['title'])
            except:
                str = str.replace('{position}',exp['position'][0]['title'])
            str = str.replace('{company}',exp['company/institute'])
            str = str.replace('{total_time}',exp['total_duration'])
        elif key == 'college':
            str = str.replace('{college}',data['education'][0]['institute'])
            str = str.replace('{degree}',data['education'][0]['degree'])
        template += str
    except:
        return createTemplate(data,strings = strings)
    
    return template
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')
print(createTemplate(shivam)+'\n')


NameError: name 'shivam' is not defined

In [24]:
list(strings.keys())

['hi',
 'location',
 'skill',
 'awards',
 'certifications',
 'experience',
 'praise',
 'praise-params']

Testing the data now

In [110]:
import scraper
import creds
url = 'https://www.linkedin.com/in/shivam-soni-b46992148/'
driver = scraper.init_driver(creds.username,creds.password)
dict = scraper.scrape_profile(url,driver)

In [22]:
import pandas as pd
table = pd.read_csv('./sample_csv/US_Architect_270_QL.csv')

url_col = '' #Find the column that stores the url

for column in table.columns:
    try:
        str = table[column][0]
        if str[:25] == 'https://www.linkedin.com/' or str[:24] == 'http://www.linkedin.com/':
            url_col = column
            break
    except:
        pass

list(table[url_col])

['http://www.linkedin.com/in/anastasia-prosina-b1354b106',
 'http://www.linkedin.com/in/aaron-sorrell-3941a67',
 'http://www.linkedin.com/in/aaron-leshtz-602b366',
 'http://www.linkedin.com/in/aaronjdemayo',
 'http://www.linkedin.com/in/ali-basbous-34298118',
 'http://www.linkedin.com/in/abigail-coover-2820429',
 'http://www.linkedin.com/in/adamarogers',
 'http://www.linkedin.com/in/adam-rosa-61a9a826',
 'http://www.linkedin.com/in/adam-steinbach-aia-leed-ap-1388a11a',
 'http://www.linkedin.com/in/adelleyork',
 'http://www.linkedin.com/in/adinahall',
 'http://www.linkedin.com/in/adi-gershoni-491861b',
 'http://www.linkedin.com/in/albert-gallo-48b53b16',
 'http://www.linkedin.com/in/anthony-tony-giorgio-87074ba',
 'http://www.linkedin.com/in/abby-gordon-967a702b',
 'http://www.linkedin.com/in/alexander-gorlin-0908528',
 'http://www.linkedin.com/in/alejandro-antonio-aguirrechu-b6112791',
 'http://www.linkedin.com/in/agustinacervera',
 'http://www.linkedin.com/in/ajtaaca',
 'http://www.li

In [27]:
import random

strings = {}



#Only use if location is provided
strings['location'] = [
    'I heard you\'re from {location}, how\'s the weather there? ',
    'I heard you\'re from {location}! Been there myself, its a lovely place. ',
    'I can see you\'re from {location}. It\'s a beautiful place. ',
]

#Only use if skill is provided
strings['skill'] = [
    'I have been looking at your profile. I believe you are skilled enough at {skill}. That must have taken a lot of hours ',
    'Looking at your profile, I could see you have a lot of experience with {skill}. How did you get so good at it?',
    'I can see that you are very skilled in {skill}. When did you start working on it?', # Multiple skills
]

strings['awards'] = [
    'Congratulations for your {award}, it is a feat not everyone can achieve... More power to you!',
    'I can also see that you have been awarded {award}! Congratulations to you! How would you describe the joy of the moment you recieved it?',
]

#Only use if certification is provided
strings['certifications'] = [
    'I can also see that you are certified in {certificate} by {issuer}. That is quite a feat! Kudos to you...',
    'I can see that you have a highly valuable {issuer} certification in {certificate}. Congratulations to you!',
]

strings['recommendations'] = [
    'I can see that you have been recommended by {giver} for your skills. That is quite a feat! Kudos to you...',
    'I can see in your profile that {giver} had quite a time working with you. I\'m sure ou must be a great person to work with',
    'I can see that you have been recommended by {giver} on your profile. You have really great connections!',
]

#Only use if experience is provided
strings['experience'] = [
    'I can see that you have experience as a {position} at {company}. How was the experience?',
    'Being a {position} at {company} must have been quite an experience for you.',
    'I can see that you have worked at {company} for the period of {total_time}. How was the experience?',
]

strings['college'] = [
    'I can see that you are an alumni of {college}. That is something to be proud of... More power to you!',
    'Being an alumni of the presitgious {college} is something that sets you apart from the crowd. Keep crushing it!',
    'It is a delight to know that you were a student of {college}. ',
    'It is a great honor to know that you did your {degree} graduated from {college}.',
]

def create_template(data,strings = strings):
    '''
    Creates a template from the strings dictionary
    '''
    random.seed()
    template = ''
    #randomly select from list
    template += random.choice(['Hi', 'Greetings', 'Hello', 'Hey']) + ' '
    template += data['name'].split(' ')[0] + ', '
    key = random.choice( list(strings.keys()) )
    print('Key: ', key, '\n')
    str = random.choice(strings[key])
    try:
        if key == 'location':
            str = str.replace('{location}',data['intro-location'].split(',')[0])
        elif key == 'skill':
            str = str.replace('{skill}',random.choice(data['skills']))
        elif key == 'awards':
            award = random.choice(data['awards'])
            str = str.replace('{award}', award['title']).replace('{issuer}', award['issuer'])
        elif key == 'recommendations':
            recc = random.choice(data['recommendations'])
            str = str.replace('{giver}',recc['giver'])
        elif key == 'certifications':
            cert = random.choice(data['licenses_and_certifications'])
            str = str.replace('{certificate}',cert['name']).replace('{issuer}',cert['issuer'])
        elif key == 'experience':
            exp = random.choice(data['experience'])
            try:
                str = str.replace('{position}',exp['position']['title'])
            except:
                str = str.replace('{position}',exp['position'][0]['title'])
            str = str.replace('{company}',exp['company/institute'])
            str = str.replace('{total_time}',exp['total_duration'])
        elif key == 'college':
            str = str.replace('{college}',data['education'][0]['institute'])
            str = str.replace('{degree}',data['education'][0]['degree'])
        template += str
    except:
        return create_template(data,strings = strings)
    
    return template,key

In [28]:
import scraper
import pandas as pd
import time
username = 'dipadey213@gmail.com'
password = 'dipa9864d$#'

driver = scraper.init_driver(username,password)

for n in range(3):
    table['Trait {}'.format(n+1)] = ''
    table['Compliment Line {}'.format(n+1)] = ''

for i in table.index:
    url = table[url_col][i] #Get the URL
    data = scraper.scrape_profile(url,driver) #Scrape the profile
    for j in range(3):
        txt,key = create_template(data)
        table['Trait {}'.format(j+1)][i] = key
        table['Compliment Line {}'.format(j+1)][i] = txt
    time.sleep(5)
    if i == 10:
        break




Key:  recommendations 

Key:  recommendations 

Key:  recommendations 



/tmp/ipykernel_131217/3639375466.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Trait {}'.format(j)][i] = key
/tmp/ipykernel_131217/3639375466.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Compliment Line {}'.format(j)][i] = txt


Key:  certifications 

Key:  recommendations 

Key:  certifications 

Key:  experience 

Key:  location 

Key:  certifications 

Key:  location 

Key:  location 

Key:  certifications 

Key:  certifications 

Key:  recommendations 

Key:  college 

Key:  location 

Key:  skill 

Key:  skill 

Key:  college 

Key:  recommendations 

Key:  awards 

Key:  awards 

Key:  location 

Key:  location 

Key:  experience 

Key:  certifications 

Key:  recommendations 

Key:  awards 

Key:  college 

Key:  skill 

Key:  awards 

Key:  certifications 

Key:  awards 

Key:  college 

Key:  skill 

Key:  skill 

Key:  awards 

Key:  skill 

Key:  certifications 

Key:  skill 

Key:  skill 



In [29]:
table.columns

Index(['Subject Line 1', 'Quicklines 1', 'Subject Line 2', 'Quicklines 2',
       'Subject Line 3', 'Quicklines 3', 'Cleaned Company Name', 'Status',
       'Error Message', 'Email', 'Verification Result', 'Verified At',
       'Syntax Error', 'ISP', 'Role', 'Disposable', 'Trap', 'First Name',
       'Last Name', 'Title', 'Company', 'Company Name for Emails',
       'Email Status', 'Email Confidence', 'Seniority', 'Departments',
       'First Phone', 'Corporate Phone', 'Lists', '# Employees', 'Industry',
       'Keywords', 'Person Linkedin Url', 'Website', 'Company Linkedin Url',
       'Facebook Url', 'Twitter Url', 'City', 'State', 'Country',
       'Company Address', 'Company City', 'Company State', 'Company Country',
       'Company Phone', 'Technologies', 'Annual Revenue', 'Trait 0',
       'Compliment Line 0', 'Trait 1', 'Compliment Line 1', 'Trait 2',
       'Compliment Line 2'],
      dtype='object')

In [34]:
table[[url_col,'Compliment Line 0','Trait 0','Compliment Line 1','Trait 1','Compliment Line 2','Trait 2']].iloc[:10].to_csv('new.csv')

In [54]:
df = pd.DataFrame()
df['Name'] = ''
dict = [
    {
        'Name' : 'Kaushik'
    },
    {
        'Name' : 'Dipadey'
    },
    {
        'Name' : 'Sachin'
    }

]
df2 = pd.DataFrame(dict)
df = pd.concat([df2,df],ignore_index=True)
df

,Name
0,Kaushik
1,Dipadey
2,Sachin
